# Reboot - SQL Advanced

Tonight we will work with the `blog.sqlite` database is available at this URL:  
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite`

In [6]:
!curl https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite > blog.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  288k  100  288k    0     0  90067      0  0:00:03  0:00:03 --:--:-- 90269


Let's have a look at our directory structure

In [7]:
!tree

.
├── README.md
├── blog.sqlite
├── data
└── recap.ipynb

2 directories, 3 files


## 1. Schema ERD

❓ Open the `data/blog.sqlite` in DBeaver, explore the schema and draw it on [kitt.lewagon.com/db](https://kitt.lewagon.com/db).

_TODO: Double click this cell and **paste** a screenshot of the schema for future reference_.

---
## 2. Most liked posts

Complete the code to get **the 3 most liked posts**:

In [37]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("data/blog.sqlite")

# Your query
query = """
SELECT p.id, p.title, COUNT(l.id) as like_count
FROM posts p
LEFT JOIN likes l ON p.id = l.post_id
GROUP BY p.id
ORDER BY like_count DESC
LIMIT 3
"""

df = pd.read_sql_query(query, conn)
print(df)

    id                      title  like_count
0  143      Half imagine another.          84
1   83  Side foot leader popular.          82
2   99  Area paper whatever mean.          81


---

### Pretty Print using _pandas_

The readbility of our `print()` statements is not so good.

Next week, we will introduce [pandas](https://pandas.pydata.org/) which will vastly improve the UX of our data exploration in Notebooks.

Execute the following cell to load `pandas` library:

In [30]:
import pandas as pd

Then try the previous `query` again, delegating the job of fetching results + displaying them to the `read_sql_query` function of `pandas`:

In [31]:
pd.read_sql_query(query, conn)

,id,title,like_count
0,143,Half imagine another.,84
1,83,Side foot leader popular.,82
2,99,Area paper whatever mean.,81


---
## 3. Find the three users who 'liked' the most

In [32]:
query = """
SELECT u.id, u.first_name, u.last_name, COUNT(l.id) AS like_count
FROM users u
JOIN likes l ON u.id = l.user_id
GROUP BY u.id
ORDER BY like_count DESC
LIMIT 3
"""

df = pd.read_sql_query(query, conn)
print(df)

   id first_name last_name  like_count
0  43    Michael     Allen         236
1  12      Donna   Ramirez         233
2  44     Hayley  Williams         227


---
## 4. Find the most liked author

In [39]:
query = """
SELECT u.id, u.first_name, u.last_name, COUNT(l.id) AS like_count
FROM users u
JOIN posts p ON u.id = p.user_id
JOIN likes l ON p.id = l.post_id
GROUP BY u.id
ORDER BY like_count DESC
LIMIT 1
"""

# Execute the query
df = pd.read_sql_query(query, conn)
print(df)

   id first_name last_name  like_count
0  57     Teresa     Moore         647


---
## 5. Who are the authors of the 3 most liked posts?

In [40]:
query = """
SELECT u.id, u.first_name, u.last_name, p.id AS post_id, COUNT(l.id) AS like_count
FROM users u
JOIN posts p ON u.id = p.user_id
JOIN likes l ON p.id = l.post_id
GROUP BY p.id, u.id
ORDER BY like_count DESC
LIMIT 3
"""

df = pd.read_sql_query(query, conn)
print(df)

   id first_name last_name  post_id  like_count
0  72    Melissa     Henry      143          84
1  63    Cynthia   Raymond       83          82
2  64  Alexander      Cook       99          81


---
## 6. How many people liked at least one post?

In [41]:
query = """
SELECT COUNT(DISTINCT user_id) AS unique_likers
FROM likes
"""
df = pd.read_sql_query(query, conn)
print(df)

   unique_likers
0             49


---
## 7. Compute the cumulative number of likes per day

In [43]:
query = """
SELECT 
    date(created_at) AS like_day, 
    COUNT(id) OVER (ORDER BY date(created_at)) AS cumulative_likes
FROM likes
GROUP BY like_day
ORDER BY like_day
"""
df = pd.read_sql_query(query, conn)
print(df)

       like_day  cumulative_likes
0    2019-01-01                 1
1    2019-01-02                 2
2    2019-01-03                 3
3    2019-01-04                 4
4    2019-01-05                 5
..          ...               ...
331  2019-12-24               332
332  2019-12-25               333
333  2019-12-26               334
334  2019-12-27               335
335  2019-12-28               336

[336 rows x 2 columns]


---
## 8. (Optional) Who's the biggest fan/ fans of each author?

The biggest fan/ fans of an author is defined as the user or users who liked the author's posts the most. i.e. if there is a tie between fans that both liked an author 20 times, both fans should be returned alongside their like count and the author in question.
<br><br>
<details>
    <summary>💡 Click for Hint</summary>
    You might need to use <code>WITH</code>
</details>


In [44]:
query = """
WITH AuthorLikes AS (
    SELECT 
        p.author_id,
        l.user_id,
        COUNT(*) AS like_count
    FROM 
        likes l
    JOIN 
        posts p ON l.post_id = p.id
    GROUP BY 
        p.author_id, l.user_id
),
MaxLikes AS (
    SELECT 
        author_id,
        MAX(like_count) AS max_like_count
    FROM 
        AuthorLikes
    GROUP BY 
        author_id
)
SELECT 
    u.id AS fan_id,
    u.first_name || ' ' || u.last_name AS fan_name,
    a.id AS author_id,
    a.first_name || ' ' || a.last_name AS author_name,
    al.like_count
FROM 
    MaxLikes ml
JOIN 
    AuthorLikes al ON ml.author_id = al.author_id AND ml.max_like_count = al.like_count
JOIN 
    users u ON al.user_id = u.id
JOIN 
    users a ON al.author_id = a.id
ORDER BY 
    al.author_id, al.like_count DESC
"""

df = pd.read_sql_query(query, conn)
print(df)

DatabaseError: Execution failed on sql '
WITH AuthorLikes AS (
    SELECT 
        p.author_id,
        l.user_id,
        COUNT(*) AS like_count
    FROM 
        likes l
    JOIN 
        posts p ON l.post_id = p.id
    GROUP BY 
        p.author_id, l.user_id
),
MaxLikes AS (
    SELECT 
        author_id,
        MAX(like_count) AS max_like_count
    FROM 
        AuthorLikes
    GROUP BY 
        author_id
)
SELECT 
    u.id AS fan_id,
    u.first_name || ' ' || u.last_name AS fan_name,
    a.id AS author_id,
    a.first_name || ' ' || a.last_name AS author_name,
    al.like_count
FROM 
    MaxLikes ml
JOIN 
    AuthorLikes al ON ml.author_id = al.author_id AND ml.max_like_count = al.like_count
JOIN 
    users u ON al.user_id = u.id
JOIN 
    users a ON al.author_id = a.id
ORDER BY 
    al.author_id, al.like_count DESC
': no such column: p.author_id